## Perform Basic MongoDB Operations Demonstrating CAP Theorem Behavior

This practical demonstrates how MongoDB handles **Consistency (C)**, **Availability (A)**, and **Partition Tolerance (P)** by setting up a Replica Set and simulating a network failure.

### **1. Install and Set Up MongoDB with Replica Sets**

We will create a 3-node Replica Set on a single machine using different ports to simulate three separate servers.

**Step A: Create Data Directories**
Create three separate folders for data storage.

```bash
mkdir -p C:\data\rs1 C:\data\rs2 C:\data\rs3

```

**Step B: Start Three Mongod Instances**
Open **three separate Command Prompt windows** and run one command in each.

* **Window 1 (Node 1 - Primary):**
```cmd
mongod --replSet myReplSet --port 27017 --dbpath C:\data\rs1 --bind_ip localhost

```


* **Window 2 (Node 2 - Secondary):**
```cmd
mongod --replSet myReplSet --port 27018 --dbpath C:\data\rs2 --bind_ip localhost

```


* **Window 3 (Node 3 - Secondary):**
```cmd
mongod --replSet myReplSet --port 27019 --dbpath C:\data\rs3 --bind_ip localhost

```



**Step C: Initialize the Replica Set**
Open a fourth window to access the shell on the first node.

```cmd
mongosh --port 27017

```

Run the initiation command in the shell:

```javascript
rs.initiate({
  _id: "myReplSet",
  members: [
    { _id: 0, host: "localhost:27017" },
    { _id: 1, host: "localhost:27018" },
    { _id: 2, host: "localhost:27019" }
  ]
})

```

*Wait until the prompt changes to `[myReplSet:PRIMARY]`.*

---

### **2. Perform Basic Write and Read Operations**

**Step A: Write Data (On Primary)**

```javascript
use capTest
db.logs.insertOne({ id: 1, status: "System Normal", timestamp: new Date() })

```

**Step B: Verify Replication (Read from Secondary)**
Open a new terminal for Node 2 (`mongosh --port 27018`) and check if data arrived.

```javascript
rs.secondaryOk()  // Allow reading from secondary
use capTest
db.logs.find()

```

*Result: You should see the record, proving **Consistency** under normal conditions.*

---

### **3. Simulate Network Disconnection (Partition)**

To test CAP behavior, we will simulate a failure by killing the Primary node.

**Action:** Go to the terminal window running **Node 1 (Port 27017)** and press `Ctrl+C` to stop it.

**Observation:**

* Watch the terminals for Node 2 and Node 3.
* You will see logs indicating "Heartbeat failed" and an **Election** process.
* One of the remaining nodes (e.g., 27018) will promote itself to **PRIMARY**.

---

### **4. Observe Read/Write Behavior During Disruption**

**Test 1: Availability (Can we still write?)**
Connect to the **new Primary** (e.g., Port 27018) and try to write.

```javascript
db.logs.insertOne({ id: 2, status: "Node 1 Down", timestamp: new Date() })

```

* **Result: Success.** The system prioritized **Availability**. It accepted the write even though one node is missing.

**Test 2: Consistency Protection (Split Brain prevention)**
Now, kill **Node 3** as well (so only Node 2 is left).

* Try to write to Node 2:
```javascript
db.logs.insertOne({ id: 3, status: "Only One Survivor" })

```


* **Result: Failed/Blocked.** Node 2 will demote itself to **SECONDARY** because it cannot see a majority (2 out of 3).
* **Conclusion:** MongoDB chooses **Consistency** over Availability when a majority cannot be established (CP behavior).

---

### **5. Record Resource Values (Observation Log)**

| State | Action | Node Targeted | Outcome | CAP Property |
| --- | --- | --- | --- | --- |
| **Normal** | Write Data | Node 1 (Pri) | Success | Normal |
| **Normal** | Read Data | Node 2 (Sec) | Success | Consistency |
| **Partition 1** (Node 1 Dead) | Write Data | Node 2 (New Pri) | **Success** | **Availability** (System stayed up) |
| **Partition 2** (Nodes 1 & 3 Dead) | Write Data | Node 2 (Survivor) | **Failed** | **Consistency** (Prevents data divergence) |

---

### **6. Sample Output (Logs)**

Save these snippets as proof of behavior.

**Election Log (Success):**

```text
I REPL [replexec] Member localhost:27017 is now in state DOWN
I REPL [replexec] Election succeeded, assuming PRIMARY role...

```

**Write Failure Log (No Majority):**

```text
WriteCommandError: not master

```